In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 模型超参数

In [105]:
# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 500
batch_size = 100
sample_dir = 'samples'

In [4]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [42]:
# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5], std=[0.5])])
                # transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))]) # 3 for RGB channels

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='./',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [43]:
# 查看图片样本
it = iter(data_loader)

In [44]:
x,_ = next(it)
x.shape

torch.Size([100, 1, 28, 28])

## 定义网络

- 问题: generator最后是否要过激活层(tanh)

In [45]:
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.leakyrelu = nn.LeakyReLU(0.2)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.leakyrelu(self.map1(x))
        x = self.leakyrelu(self.map2(x))
        x = self.sigmoid(self.map3(x))# 最后生成的是概率
        return x

In [46]:
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh() # 激活函数
        
    def forward(self, x):
        x = self.relu(self.map1(x))
        x = self.relu(self.map2(x))
        x = self.tanh(self.map3(x))
        return x

In [106]:
# ----------
# 初始化网络
# ----------
D = Discriminator(input_size=image_size,
                  hidden_size=hidden_size,
                  output_size=1).to(device)


G = Generator(input_size=latent_size,
              hidden_size=hidden_size,
              output_size=image_size).to(device)

In [96]:
# 定义辅助函数
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [107]:
# 定义损失函数和优化器
learning_rate = 0.0003
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate)
g_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate)

d_exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(d_optimizer, step_size=50, gamma=0.9)
g_exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(g_optimizer, step_size=50, gamma=0.9)

## 开始训练

- 查看fake_score, real_score的值

In [108]:
total_step = len(data_loader)
# ------------------
# 一开始学习率快一些
# ------------------
for epoch in range(250):
    d_exp_lr_scheduler.step()
    g_exp_lr_scheduler.step()
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        # 创造real label和fake label
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        # ---------------------
        # 开始训练discriminator
        # ---------------------
        
        # 首先计算真实的图片
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs # 真实图片的分类结果, 越接近1越好
        
        # 接着使用生成器训练得到图片, 放入判别器
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs # 错误图片的分类结果, 越接近0越好, 最后会趋于1, 生成器生成的判别器判断不了
        
        # 两个loss相加, 反向传播进行优化
        d_loss = d_loss_real + d_loss_fake
        g_optimizer.zero_grad() # 两个优化器梯度都要清0
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # -----------------
        # 开始训练generator
        # -----------------
        
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        g_loss = criterion(outputs, real_labels) # 希望生成器生成的图片判别器可以判别为真
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}, d_lr={:.6f},g_lr={:.6f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item(),
                         d_optimizer.param_groups[0]['lr'], g_optimizer.param_groups[0]['lr']))
        
        # Save real images
        if (epoch+1) == 1:
            images = images.reshape(images.size(0), 1, 28, 28)
            save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
        # Save sampled images
        fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
        save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# ----------------
# 之后学习率慢一些
# ----------------
d_optimizer.param_groups[0]['initial_lr'] = d_optimizer.param_groups[0]['lr']
g_optimizer.param_groups[0]['initial_lr'] = g_optimizer.param_groups[0]['lr']

d_cos_lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(d_optimizer, T_max = 10, eta_min=0.00001)
g_cos_lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(g_optimizer, T_max = 10, eta_min=0.00001)
for epoch in range(250, 500):
    d_cos_lr_scheduler.step()
    g_cos_lr_scheduler.step()
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        # 创造real label和fake label
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        # ---------------------
        # 开始训练discriminator
        # ---------------------
        
        # 首先计算真实的图片
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs # 真实图片的分类结果, 越接近1越好
        
        # 接着使用生成器训练得到图片, 放入判别器
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs # 错误图片的分类结果, 越接近0越好, 最后会趋于1, 生成器生成的判别器判断不了
        
        # 两个loss相加, 反向传播进行优化
        d_loss = d_loss_real + d_loss_fake
        g_optimizer.zero_grad() # 两个优化器梯度都要清0
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # -----------------
        # 开始训练generator
        # -----------------
        
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        g_loss = criterion(outputs, real_labels) # 希望生成器生成的图片判别器可以判别为真
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}, d_lr={:.6f},g_lr={:.6f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item(),
                         d_optimizer.param_groups[0]['lr'], g_optimizer.param_groups[0]['lr']))
        
        # Save real images
        if (epoch+1) == 1:
            images = images.reshape(images.size(0), 1, 28, 28)
            save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
        # Save sampled images
        fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
        save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
        

# Save the model checkpoints 
torch.save(G.state_dict(), './models/G.ckpt')
torch.save(D.state_dict(), './models/D.ckpt')

Epoch [0/500], Step [200/600], d_loss: 0.0432, g_loss: 4.8757, D(x): 0.98, D(G(z)): 0.03, d_lr=0.000300,g_lr=0.000300
Epoch [0/500], Step [400/600], d_loss: 0.0080, g_loss: 7.9112, D(x): 1.00, D(G(z)): 0.00, d_lr=0.000300,g_lr=0.000300
Epoch [0/500], Step [600/600], d_loss: 0.0980, g_loss: 4.3099, D(x): 0.97, D(G(z)): 0.06, d_lr=0.000300,g_lr=0.000300
Epoch [1/500], Step [200/600], d_loss: 0.1361, g_loss: 5.7767, D(x): 0.96, D(G(z)): 0.08, d_lr=0.000300,g_lr=0.000300
Epoch [1/500], Step [400/600], d_loss: 1.3944, g_loss: 5.8083, D(x): 0.86, D(G(z)): 0.54, d_lr=0.000300,g_lr=0.000300
Epoch [1/500], Step [600/600], d_loss: 0.5025, g_loss: 4.9865, D(x): 0.92, D(G(z)): 0.22, d_lr=0.000300,g_lr=0.000300
Epoch [2/500], Step [200/600], d_loss: 0.8335, g_loss: 2.8011, D(x): 0.75, D(G(z)): 0.23, d_lr=0.000300,g_lr=0.000300
Epoch [2/500], Step [400/600], d_loss: 0.1467, g_loss: 4.2990, D(x): 0.97, D(G(z)): 0.09, d_lr=0.000300,g_lr=0.000300
Epoch [2/500], Step [600/600], d_loss: 1.1007, g_loss: 2

Epoch [23/500], Step [200/600], d_loss: 0.2465, g_loss: 4.3979, D(x): 0.88, D(G(z)): 0.05, d_lr=0.000300,g_lr=0.000300
Epoch [23/500], Step [400/600], d_loss: 0.5402, g_loss: 4.3046, D(x): 0.95, D(G(z)): 0.27, d_lr=0.000300,g_lr=0.000300
Epoch [23/500], Step [600/600], d_loss: 0.7319, g_loss: 2.9315, D(x): 0.78, D(G(z)): 0.13, d_lr=0.000300,g_lr=0.000300
Epoch [24/500], Step [200/600], d_loss: 0.3815, g_loss: 3.6926, D(x): 0.85, D(G(z)): 0.07, d_lr=0.000300,g_lr=0.000300
Epoch [24/500], Step [400/600], d_loss: 0.4504, g_loss: 3.6485, D(x): 0.85, D(G(z)): 0.09, d_lr=0.000300,g_lr=0.000300
Epoch [24/500], Step [600/600], d_loss: 0.5504, g_loss: 2.7633, D(x): 0.92, D(G(z)): 0.24, d_lr=0.000300,g_lr=0.000300
Epoch [25/500], Step [200/600], d_loss: 0.2263, g_loss: 3.9623, D(x): 0.92, D(G(z)): 0.09, d_lr=0.000300,g_lr=0.000300
Epoch [25/500], Step [400/600], d_loss: 0.6472, g_loss: 3.4207, D(x): 0.77, D(G(z)): 0.05, d_lr=0.000300,g_lr=0.000300
Epoch [25/500], Step [600/600], d_loss: 0.3685, 

Epoch [45/500], Step [600/600], d_loss: 0.5694, g_loss: 2.8130, D(x): 0.79, D(G(z)): 0.12, d_lr=0.000300,g_lr=0.000300
Epoch [46/500], Step [200/600], d_loss: 0.6756, g_loss: 2.3065, D(x): 0.90, D(G(z)): 0.31, d_lr=0.000300,g_lr=0.000300
Epoch [46/500], Step [400/600], d_loss: 0.4413, g_loss: 3.7165, D(x): 0.88, D(G(z)): 0.14, d_lr=0.000300,g_lr=0.000300
Epoch [46/500], Step [600/600], d_loss: 0.4775, g_loss: 2.6447, D(x): 0.84, D(G(z)): 0.16, d_lr=0.000300,g_lr=0.000300
Epoch [47/500], Step [200/600], d_loss: 0.4467, g_loss: 2.3514, D(x): 0.85, D(G(z)): 0.17, d_lr=0.000300,g_lr=0.000300
Epoch [47/500], Step [400/600], d_loss: 0.7094, g_loss: 3.2009, D(x): 0.77, D(G(z)): 0.17, d_lr=0.000300,g_lr=0.000300
Epoch [47/500], Step [600/600], d_loss: 0.7041, g_loss: 3.0588, D(x): 0.90, D(G(z)): 0.30, d_lr=0.000300,g_lr=0.000300
Epoch [48/500], Step [200/600], d_loss: 0.5980, g_loss: 3.0996, D(x): 0.85, D(G(z)): 0.24, d_lr=0.000300,g_lr=0.000300
Epoch [48/500], Step [400/600], d_loss: 0.4918, 

Epoch [68/500], Step [400/600], d_loss: 1.0143, g_loss: 2.4632, D(x): 0.69, D(G(z)): 0.28, d_lr=0.000270,g_lr=0.000270
Epoch [68/500], Step [600/600], d_loss: 0.6655, g_loss: 1.7792, D(x): 0.84, D(G(z)): 0.27, d_lr=0.000270,g_lr=0.000270
Epoch [69/500], Step [200/600], d_loss: 0.8124, g_loss: 2.2484, D(x): 0.67, D(G(z)): 0.15, d_lr=0.000270,g_lr=0.000270
Epoch [69/500], Step [400/600], d_loss: 0.5970, g_loss: 1.9140, D(x): 0.85, D(G(z)): 0.26, d_lr=0.000270,g_lr=0.000270
Epoch [69/500], Step [600/600], d_loss: 0.6924, g_loss: 2.4424, D(x): 0.78, D(G(z)): 0.24, d_lr=0.000270,g_lr=0.000270
Epoch [70/500], Step [200/600], d_loss: 0.5526, g_loss: 2.2811, D(x): 0.81, D(G(z)): 0.18, d_lr=0.000270,g_lr=0.000270
Epoch [70/500], Step [400/600], d_loss: 0.6100, g_loss: 2.0004, D(x): 0.75, D(G(z)): 0.15, d_lr=0.000270,g_lr=0.000270
Epoch [70/500], Step [600/600], d_loss: 0.7456, g_loss: 2.3438, D(x): 0.77, D(G(z)): 0.27, d_lr=0.000270,g_lr=0.000270
Epoch [71/500], Step [200/600], d_loss: 0.7610, 

Epoch [91/500], Step [200/600], d_loss: 0.9136, g_loss: 2.1543, D(x): 0.77, D(G(z)): 0.32, d_lr=0.000270,g_lr=0.000270
Epoch [91/500], Step [400/600], d_loss: 0.6275, g_loss: 1.8625, D(x): 0.76, D(G(z)): 0.20, d_lr=0.000270,g_lr=0.000270
Epoch [91/500], Step [600/600], d_loss: 0.6982, g_loss: 2.2900, D(x): 0.77, D(G(z)): 0.22, d_lr=0.000270,g_lr=0.000270
Epoch [92/500], Step [200/600], d_loss: 0.7757, g_loss: 2.4725, D(x): 0.70, D(G(z)): 0.20, d_lr=0.000270,g_lr=0.000270
Epoch [92/500], Step [400/600], d_loss: 0.8216, g_loss: 2.1354, D(x): 0.74, D(G(z)): 0.25, d_lr=0.000270,g_lr=0.000270
Epoch [92/500], Step [600/600], d_loss: 0.8639, g_loss: 1.8377, D(x): 0.79, D(G(z)): 0.34, d_lr=0.000270,g_lr=0.000270
Epoch [93/500], Step [200/600], d_loss: 0.7639, g_loss: 1.8796, D(x): 0.71, D(G(z)): 0.19, d_lr=0.000270,g_lr=0.000270
Epoch [93/500], Step [400/600], d_loss: 0.8652, g_loss: 1.8265, D(x): 0.70, D(G(z)): 0.25, d_lr=0.000270,g_lr=0.000270
Epoch [93/500], Step [600/600], d_loss: 0.7186, 

...

Epoch [408/500], Step [400/600], d_loss: 0.8363, g_loss: 1.7791, D(x): 0.72, D(G(z)): 0.26, d_lr=0.000179,g_lr=0.000179
Epoch [408/500], Step [600/600], d_loss: 0.8050, g_loss: 1.5535, D(x): 0.72, D(G(z)): 0.28, d_lr=0.000179,g_lr=0.000179
Epoch [409/500], Step [200/600], d_loss: 0.9042, g_loss: 1.8806, D(x): 0.68, D(G(z)): 0.25, d_lr=0.000192,g_lr=0.000192
Epoch [409/500], Step [400/600], d_loss: 1.0442, g_loss: 1.4389, D(x): 0.79, D(G(z)): 0.42, d_lr=0.000192,g_lr=0.000192
Epoch [409/500], Step [600/600], d_loss: 0.9548, g_loss: 1.6804, D(x): 0.70, D(G(z)): 0.30, d_lr=0.000192,g_lr=0.000192
Epoch [410/500], Step [200/600], d_loss: 1.0725, g_loss: 1.2430, D(x): 0.77, D(G(z)): 0.39, d_lr=0.000197,g_lr=0.000197
Epoch [410/500], Step [400/600], d_loss: 0.8983, g_loss: 1.2662, D(x): 0.74, D(G(z)): 0.30, d_lr=0.000197,g_lr=0.000197
Epoch [410/500], Step [600/600], d_loss: 1.0301, g_loss: 1.6591, D(x): 0.65, D(G(z)): 0.32, d_lr=0.000197,g_lr=0.000197
Epoch [411/500], Step [200/600], d_loss:

Epoch [431/500], Step [200/600], d_loss: 0.8347, g_loss: 1.6333, D(x): 0.71, D(G(z)): 0.26, d_lr=0.000192,g_lr=0.000192
Epoch [431/500], Step [400/600], d_loss: 0.9252, g_loss: 1.5289, D(x): 0.73, D(G(z)): 0.33, d_lr=0.000192,g_lr=0.000192
Epoch [431/500], Step [600/600], d_loss: 0.7651, g_loss: 1.9978, D(x): 0.74, D(G(z)): 0.23, d_lr=0.000192,g_lr=0.000192
Epoch [432/500], Step [200/600], d_loss: 0.7641, g_loss: 1.7249, D(x): 0.73, D(G(z)): 0.26, d_lr=0.000179,g_lr=0.000179
Epoch [432/500], Step [400/600], d_loss: 0.9247, g_loss: 1.7592, D(x): 0.62, D(G(z)): 0.22, d_lr=0.000179,g_lr=0.000179
Epoch [432/500], Step [600/600], d_loss: 0.8959, g_loss: 1.5727, D(x): 0.74, D(G(z)): 0.31, d_lr=0.000179,g_lr=0.000179
Epoch [433/500], Step [200/600], d_loss: 0.8139, g_loss: 1.8768, D(x): 0.67, D(G(z)): 0.20, d_lr=0.000158,g_lr=0.000158
Epoch [433/500], Step [400/600], d_loss: 0.8313, g_loss: 1.4887, D(x): 0.72, D(G(z)): 0.30, d_lr=0.000158,g_lr=0.000158
Epoch [433/500], Step [600/600], d_loss:

Epoch [453/500], Step [600/600], d_loss: 0.9095, g_loss: 1.8565, D(x): 0.68, D(G(z)): 0.28, d_lr=0.000158,g_lr=0.000158
Epoch [454/500], Step [200/600], d_loss: 0.7593, g_loss: 1.5356, D(x): 0.76, D(G(z)): 0.28, d_lr=0.000132,g_lr=0.000132
Epoch [454/500], Step [400/600], d_loss: 0.8278, g_loss: 1.8551, D(x): 0.73, D(G(z)): 0.29, d_lr=0.000132,g_lr=0.000132
Epoch [454/500], Step [600/600], d_loss: 0.7660, g_loss: 1.7213, D(x): 0.77, D(G(z)): 0.32, d_lr=0.000132,g_lr=0.000132
Epoch [455/500], Step [200/600], d_loss: 0.8237, g_loss: 1.6693, D(x): 0.75, D(G(z)): 0.30, d_lr=0.000103,g_lr=0.000103
Epoch [455/500], Step [400/600], d_loss: 0.8233, g_loss: 1.6093, D(x): 0.74, D(G(z)): 0.29, d_lr=0.000103,g_lr=0.000103
Epoch [455/500], Step [600/600], d_loss: 0.8526, g_loss: 1.6719, D(x): 0.69, D(G(z)): 0.25, d_lr=0.000103,g_lr=0.000103
Epoch [456/500], Step [200/600], d_loss: 0.8435, g_loss: 1.8559, D(x): 0.69, D(G(z)): 0.26, d_lr=0.000075,g_lr=0.000075
Epoch [456/500], Step [400/600], d_loss:

Epoch [476/500], Step [400/600], d_loss: 0.8091, g_loss: 1.7846, D(x): 0.71, D(G(z)): 0.26, d_lr=0.000075,g_lr=0.000075
Epoch [476/500], Step [600/600], d_loss: 0.7258, g_loss: 1.6150, D(x): 0.74, D(G(z)): 0.26, d_lr=0.000075,g_lr=0.000075
Epoch [477/500], Step [200/600], d_loss: 0.7756, g_loss: 1.5636, D(x): 0.71, D(G(z)): 0.24, d_lr=0.000049,g_lr=0.000049
Epoch [477/500], Step [400/600], d_loss: 0.9026, g_loss: 1.6546, D(x): 0.73, D(G(z)): 0.32, d_lr=0.000049,g_lr=0.000049
Epoch [477/500], Step [600/600], d_loss: 0.6838, g_loss: 1.3601, D(x): 0.78, D(G(z)): 0.28, d_lr=0.000049,g_lr=0.000049
Epoch [478/500], Step [200/600], d_loss: 0.8448, g_loss: 1.7378, D(x): 0.69, D(G(z)): 0.25, d_lr=0.000028,g_lr=0.000028
Epoch [478/500], Step [400/600], d_loss: 0.7800, g_loss: 1.5049, D(x): 0.74, D(G(z)): 0.29, d_lr=0.000028,g_lr=0.000028
Epoch [478/500], Step [600/600], d_loss: 0.8013, g_loss: 1.5762, D(x): 0.75, D(G(z)): 0.30, d_lr=0.000028,g_lr=0.000028
Epoch [479/500], Step [200/600], d_loss:

Epoch [499/500], Step [200/600], d_loss: 0.7306, g_loss: 1.7295, D(x): 0.73, D(G(z)): 0.23, d_lr=0.000015,g_lr=0.000015
Epoch [499/500], Step [400/600], d_loss: 0.8559, g_loss: 1.6985, D(x): 0.71, D(G(z)): 0.27, d_lr=0.000015,g_lr=0.000015
Epoch [499/500], Step [600/600], d_loss: 0.7616, g_loss: 1.8125, D(x): 0.75, D(G(z)): 0.28, d_lr=0.000015,g_lr=0.000015
